<a href="https://colab.research.google.com/github/Ankan1998/Chatbot-for-Transport-Rental-Agency/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **nltk_utils.py**
#### This includes nltk package

In [10]:
import numpy as np
import nltk
nltk.download('punkt')


from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

def token_ize(sentence):
    # tokenize each sentence by splitting from " " 
    return nltk.word_tokenize(sentence)


def stemming(word):
    return stemmer.stem(word.lower())

def b_o_w(tokenize_sentence,list_of_words):
    """
    Parameters
    ----------
    tokenize_sentence : ["hi","how","are","you"]
        result of tokenizer 
    list_of_words : ["hi","bye","how","are","you"]
        bag of all words
    Returns
    -------
    list with [1,0,1,1,1]
    """
    stemmed=[stemming(w) for w in tokenize_sentence]
    bag=np.zeros(len(list_of_words),dtype=np.float32)
    for idx,word in enumerate(list_of_words):
        if word in stemmed:
            bag[idx]=1.0
    return bag


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### **model.py**

In [14]:
import torch
import torch.nn as nn

class ChatNet(nn.Module):
  def __init__(self,input_size,classes):
    super().__init__()
    self.l1=nn.Linear(input_size,128)
    self.l2=nn.Linear(128,64)
    self.l3=nn.Linear(64,32)
    self.l4=nn.Linear(32,classes)
    self.d0=nn.Dropout(p=0.1)
    self.relu=nn.ReLU()


  def forward(self,x):
    out=self.l1(x)
    out=self.relu(out)
    out=self.l2(out)
    out=self.relu(out)
    out=self.l3(out)
    out=self.relu(out)
    out=self.d0(out)
    out=self.l4(out)

    return out


### **train.py**
#### These include dataset, dataloader and neural trainer, model saver

In [15]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

with open("/content/drive/MyDrive/PROJECTS/Chatbot Transport Rental Agency/intents.json","r") as f:
  intents = json.load(f)


# Separating required field from dictionary
all_words=[]
tags=[]
x_y=[]

for intent in intents["intents"]:
  t=intent["tag"]
  tags.append(t)
  for i in intent["patterns"]:
    each_word=token_ize(i)
    all_words.extend(each_word)
    x_y.append((each_word,t))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stemming(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))


# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in x_y:
    # X: bag of words for each pattern_sentence
    bag = b_o_w(pattern_sentence, all_words)
    X_train.append(bag)
    # y: Label index
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)



# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.0001
input_size = len(X_train[0])
output_size = len(tags)
print(input_size, output_size)



class Chat_Dataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples



dataset = Chat_Dataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ChatNet(input_size, output_size).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  for words,labels in train_loader:

    words=words.to(device)
    labels = labels.to(dtype=torch.long).to(device)

    outputs = model(words)

    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "/content/drive/MyDrive/PROJECTS/Chatbot Transport Rental Agency/data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


79 13
Epoch [100/1000], Loss: 2.2432
Epoch [100/1000], Loss: 1.8298
Epoch [100/1000], Loss: 2.2853
Epoch [100/1000], Loss: 2.0390
Epoch [100/1000], Loss: 2.0309
Epoch [100/1000], Loss: 2.1714
Epoch [200/1000], Loss: 1.0066
Epoch [200/1000], Loss: 0.7666
Epoch [200/1000], Loss: 0.7188
Epoch [200/1000], Loss: 0.9125
Epoch [200/1000], Loss: 0.7478
Epoch [200/1000], Loss: 1.1986
Epoch [300/1000], Loss: 0.1701
Epoch [300/1000], Loss: 0.2732
Epoch [300/1000], Loss: 0.2985
Epoch [300/1000], Loss: 0.2694
Epoch [300/1000], Loss: 0.1937
Epoch [300/1000], Loss: 0.0673
Epoch [400/1000], Loss: 0.1875
Epoch [400/1000], Loss: 0.0940
Epoch [400/1000], Loss: 0.0479
Epoch [400/1000], Loss: 0.0374
Epoch [400/1000], Loss: 0.0840
Epoch [400/1000], Loss: 0.1704
Epoch [500/1000], Loss: 0.0153
Epoch [500/1000], Loss: 0.0544
Epoch [500/1000], Loss: 0.0338
Epoch [500/1000], Loss: 0.0587
Epoch [500/1000], Loss: 0.0673
Epoch [500/1000], Loss: 0.0640
Epoch [600/1000], Loss: 0.0420
Epoch [600/1000], Loss: 0.0625
Ep

### **Chat.py**

In [16]:
import random
import json

import torch

#from model import NeuralNet
#from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('/content/drive/MyDrive/PROJECTS/Chatbot Transport Rental Agency/intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "/content/drive/MyDrive/PROJECTS/Chatbot Transport Rental Agency/data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = ChatNet(input_size,output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sammy"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = token_ize(sentence)
    X = b_o_w(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.6:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hey
Sammy: Hello, thanks for visiting
You: when are you guys open
Sammy: We're open every day 9am-9pm
You: what about today
Sammy: Same-day rentals please call 1-800-carbike
You: what transports are avialble
Sammy: we rent cars and bikes
You: what cars
Sammy: We have jeep, sedan and taxi
You: what is rental policy
Sammy: We have two ways of rental policy, days or distance travelled
You: what about distance
Sammy: Rs 200 per 10 k.m, isnt it cheap?
You: ok how can i rent
Sammy: For rentals today please call 1-800-carbike
You: thanks man
Sammy: My pleasure
You: quit
